# Modèle basé sur les données manuellement étiquetées

L'objet de ce notebook est de mettre en place le modèle basé sur les données manuellement étiquetées.

## Récupération des données

### Préambule technique



In [1]:
# setting up sys.path for relative imports
from pathlib import Path
import sys
project_root = str(Path(sys.path[0]).parents[1].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# imports and customization of diplay
import os
import pandas as pd
pd.options.display.min_rows = 6
pd.options.display.width = 108
pd.options.display.latex.repr = True
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from src.pimapi import Requester
from src.pimest import ContentGetter
from src.pimest import PathGetter
from src.pimest import PDFContentParser
from src.pimest import BlockSplitter
from src.pimest import SimilaritySelector

### Chargement du fichier des données manuellement étiquetées

On commence par charger le fichier csv contenant les données manuellement étiquetées.

In [3]:
ground_truth_df = pd.read_csv(Path('..') / '..' / 'ground_truth' / 'manually_labelled_ground_truth.csv',
                              sep=';',
                              encoding='latin-1',
                              index_col='uid')
ground_truth_df.head()

,designation,ingredients
uid,,
a0492df6-9c76-4303-8813-65ec5ccbfa70,Concentré liquide Asian en bouteille 980 ml CHEF,"Eau, maltodextrine, sel, arômes, sucre, arôme ..."
d183e914-db2f-4e2f-863a-a3b2d054c0b8,Pain burger curry 80 g CREATIV BURGER,"Farine de blé T65, eau, levure, vinaigre de ci..."
ab48a1ed-7a3d-4686-bb6d-ab4f367cada8,Macaroni en sachet 500 g PANZANI,- 100% Semoule de BLE dur de qualité supérieur...
528d4be3-425c-4f8b-8a87-12f1bc645ddd,Fève de Tonka en sachet 100 g COMPTOIR COLONIAL,fève de tonka (graines ridées de 25 à 50mm de ...
51b38427-b2ea-4c56-93e8-4242361ef31b,Caviar d'aubergine en pot 500 g PUGET RESTAURA...,"Aubergine 60,5% (aubergine, huile de tournesol..."


In [4]:
ground_truth_uids = list(ground_truth_df.index)
ground_truth_uids[:5]

['a0492df6-9c76-4303-8813-65ec5ccbfa70',
 'd183e914-db2f-4e2f-863a-a3b2d054c0b8',
 'ab48a1ed-7a3d-4686-bb6d-ab4f367cada8',
 '528d4be3-425c-4f8b-8a87-12f1bc645ddd',
 '51b38427-b2ea-4c56-93e8-4242361ef31b']

### Pipeline d'acquisition du contenu des données

On commence par construire un premier pipeline d'acquisition des données.
Il fonctionne en 3 étapes : 
- détermination du chemin vers lequel aller chercher les fiches techniques
- récupération du contenu binaire du fichier
- conversion de ce contenu binaire en texte

In [5]:
acqui_pipe = Pipeline([('PathGetter', PathGetter(ground_truth_uids=ground_truth_uids,
                                                  train_set_path=Path('..') / '..' / 'ground_truth',
                                                  ground_truth_path=Path('..') / '..' / 'ground_truth',
                                                  )),
                        ('ContentGetter', ContentGetter(missing_file='to_nan')),
                        ('ContentParser', PDFContentParser(none_content='to_empty')),
                       ],
                       verbose=True)

In [6]:
texts_df = acqui_pipe.fit_transform(ground_truth_df)
texts_df.sample(5)

[Pipeline] ........ (step 1 of 3) Processing PathGetter, total=   0.1s
[Pipeline] ..... (step 2 of 3) Processing ContentGetter, total=   0.1s
Launching 8 processes.
[Pipeline] ..... (step 3 of 3) Processing ContentParser, total=  36.8s


,designation,ingredients,path,content,text
uid,,,,,
92dc0ce4-d80e-4dfc-ac5e-536124871c5b,Sauce hamburger en pot 3 L COLONA,"Huile de colza (70.45 %), jaunes d'oeufs (6.2 ...",../../ground_truth/92dc0ce4-d80e-4dfc-ac5e-536...,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,\n\n \nCOORDONNEES DU FOURNISSEUR : COLONA S....
7f806c88-e985-4b3a-b1af-f349374c5a71,Velouté de tomates en boîte 925 g KNORR,"Amidon (maïs, pomme de terre), légumes (tomate...",../../ground_truth/7f806c88-e985-4b3a-b1af-f34...,b'%PDF-1.5\r%\xe2\xe3\xcf\xd3\r\n285 0 obj\r<<...,Velouté de Tomates\n\nKnorr Potage de Toujours...
2e8f2aff-58b1-4625-a754-7b4c599cc443,Jus de veau lié hyposodé à froid en poche 1 kg...,Fécule de pomme de terre - Amidon modifié de p...,../../ground_truth/2e8f2aff-58b1-4625-a754-7b4...,b'%PDF-1.3 \n%\xe2\xe3\xcf\xd3 \n1 0 obj \n<< ...,JUS DE VEAU LIÉ HYPOSODÉ À FROID\n\nFond de sa...
6566e18d-3bdd-43d8-ab0c-de51894621f9,SPÉCIALITÉ POMME-FRAISE ALLÉGÉE EN SUCRES EN B...,"Pommes 89%, purée de fraises à base de concent...",../../ground_truth/6566e18d-3bdd-43d8-ab0c-de5...,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,Les Moulinés de Fruits\n\nPOMME-FRAISE\n\nBoît...
a5d7bedc-2df0-4a84-a5c9-ec6c8294fa84,"BISCUIT SPÉCULOOS EN ÉTUI 6,25 G LOTUS 1PX50 (X8)","Farine de blé, sucre, huiles végétales (huile ...",../../ground_truth/a5d7bedc-2df0-4a84-a5c9-ec6...,b'%PDF-1.7\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,\n\n \n\nPRODUCT SPECIFICATION \n\nEU \n\n Pr...


On peut afficher quelques textes récupérés par le pipeline :

In [7]:
with pd.option_context("max_colwidth", 1000):
    pass
#     print(texts_df.sample(3, random_state=42)['text'])
#    (texts_df.sample(3, random_state=42)['text']
#             .to_latex(Path('..') / 'tbls' / 'processed_FT.tex',
#                       index=False,
#                       index_names=False,
#                       column_format='p{\linewidth}',
#                       na_rep='-',
#                       escape=True,
#                      )
#    )

## Découpage en blocs

On découpe les longs textes en blocs. Chaque texte devient une liste de strings plus court.

In [8]:
def splitter(text):
    return(text.split('\n\n'))

In [9]:
split_transfo = BlockSplitter(splitter_func=splitter)
splitted_df = split_transfo.fit_transform(texts_df)
splitted_df.sample(5)

Launching 8 processes.


,designation,ingredients,path,content,text,blocks
uid,,,,,,
eb8f42a8-8322-4aa9-874e-a28195583eb7,Filet de maquereau à la tomate en boîte 4/4 KE...,Filets de maquereaux scomber scombrus 60% (Pêc...,../../ground_truth/eb8f42a8-8322-4aa9-874e-a28...,b'%PDF-1.4\r%\xe2\xe3\xcf\xd3\r\n7 0 obj\r<</L...,\n\n \n\nGAMME \n\nDENOMINATION \n\nFORMAT \n...,"[ , , GAMME , DENOMINATION , FORMAT , , , R..."
c4afd49b-aac9-4071-a168-d0ba79ab93e6,Confiture extra de fraise en pot verre 30 g BO...,"Fraises , sucre, sucre roux de canne, jus de c...",../../ground_truth/c4afd49b-aac9-4071-a168-d0b...,b'%PDF-1.4\r%\xe2\xe3\xcf\xd3\r\n7 0 obj\r<</L...,Route de Oinville\n28 700 AUNEAU\nTél : + 33 2...,[Route de Oinville\n28 700 AUNEAU\nTél : + 33 ...
a24f9658-9ae0-4212-8f49-78124a9917c6,Saveurs des potages en bouteille 1 L PATRELLE,Colorant : Caramel de sulfite caustique E150b ...,../../ground_truth/a24f9658-9ae0-4212-8f49-781...,b'%PDF-1.4\r%\xe2\xe3\xcf\xd3\r\n12 0 obj\r<</...,,[ ]
3f62240a-8907-47b0-9483-d4a0fb796bf7,Mélange sel poivre en sachet 1 kg DUCROS,"Sel (90%), poivre noir (10%).",../../ground_truth/3f62240a-8907-47b0-9483-d4a...,b'%PDF-1.5\r%\xe2\xe3\xcf\xd3\r\n874 0 obj\r<<...,"McCormick 315, Rue Marcel Demonque – Site Agr...","[McCormick 315, Rue Marcel Demonque – Site Ag..."
321363df-147c-4e4d-bf1a-c982c0ce9951,Brandade de morue en boîte 5/1 RAYMOND GEOFFROY,"Huile de colza, morue (32%), eau, protéines de...",../../ground_truth/321363df-147c-4e4d-bf1a-c98...,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,\n \n\n \n\n \n\n \nF ICHE DE SPECIFICAT...,"[ \n , , , \nF ICHE DE SPECIFICATION ,..."


On peut afficher un exemple de texte découpé en blocs :

In [10]:
sep = '\n-----------------------------------------------------------\n'
sample = splitted_df.sample(1, random_state=39)['blocks'].iloc[0]
print(sep.join(sample))

tex_str = (
pd.DataFrame(sample, columns=['Blocs'])
  .to_latex(column_format='p{10cm}',
            index=False,
            index_names=False,
            escape=True,
           )
  .replace(r'\textbackslash n', '\\newline ')
)

#with open(Path('..') / 'tbls' / 'block_example.tex', mode='w') as file:
#    file.write(sep.join(sample).replace('\n', r' \newline '))

30/12/19
-----------------------------------------------------------
Date d'impression : 
Remarque : 
Les informations contenues dans cette fiche technique sont données de bonne foi, en l’état actuel de nos connaissances, et selon 
les indications communiquées par le producteur ou le fournisseur. Il appartient au client de vérifier la conformité de la marchandise 
par rapport à l’usage qu’il en fait.
-----------------------------------------------------------
Création : 
-----------------------------------------------------------
12/06/12
-----------------------------------------------------------
12 rue René Cassin
37390 NOTRE DAME
-----------------------------------------------------------
Tél :
02 47 85 55 00
Fax :02 47 41 33 32
-----------------------------------------------------------
FICHE TECHNIQUE
-----------------------------------------------------------
Mélange du trappeur, 70 g
Trapper blend, 70g
-----------------------------------------------------------
Code article KERE

## Train / Test split

On procède au découpage en un jeu d'entrainement et un jeu de test en gardant 400 produits pour l'entrainement et 100 produits pour le test : 

In [11]:
train, test = train_test_split(splitted_df, train_size=400, random_state=42)

## Entraînement sur le jeu d'entrainement

On entraîne un modèle `SimilaritySelector`, sur le set d'entraînement :

In [12]:
model = SimilaritySelector()

In [13]:
model.fit(train['blocks'], train['ingredients'])

In [14]:
len(model.count_vect.vocabulary_)

1204

In [15]:
predicted = pd.Series(model.predict(test['blocks']),
                      index=test.index,
                      name='predicted'
                     )
predicted = pd.concat([test['ingredients'], predicted], axis=1)
predicted.sample(5)

,ingredients,predicted
uid,,
8278ffc7-c82b-44b9-9e4d-9150943faaf1,"Tofu* 76.6% (eau, soja* dépelliculé 20.7%, gél...",Si le produit est vendu en Espagne ou en Ital...
7f622727-e4ad-45cc-9af4-4509acf91154,"Eau, huile de tournesol, beurre 9,5 %, jaune d...",Une sauce onctueuse et savoureuse avec un équi...
13980d31-9002-457d-8d49-b451f08f473c,"Edulcorants sorbitol, isomalt, sirop de maltit...","Edulcorants sorbitol, isomalt, sirop de maltit..."
5cee689e-6fb1-493c-b232-1d8fb1f88a57,"Flageolets verts. Jus : eau, sel, affermissant...","Flageolets verts. Jus : eau, sel, affermissant..."
272246a4-d79d-47fc-a475-7b873b6bb47f,"Maltodextrine, édulcorant : sucralose (1,23%).","liste des ingrédients :\nMaltodextrine, édulco..."


In [16]:
predicted['pred_len'] = predicted['predicted'].apply(len)
sub_sample = predicted.loc[predicted['pred_len'] <= 500, ['ingredients', 'predicted']]
sub_sample.head(5)

,ingredients,predicted
uid,,
2892dd68-e3a6-474c-b543-3ebfd3490658,"Café instantané, café torrefié moulu (3%).",- NESTLÉ a un système de management de la qual...
3634fb1e-ee79-41d1-8aaa-084c1fae5bd5,"Poire 99,9%, antioxydant: acide ascorbique.",Ce produit est une purée de fruits obtenue à p...
345591f4-d887-4ddc-bb40-21337fa9269d,"Gésier de dinde émincé 50%, graisse de canard ...","Gésier de dinde émincé 50%, graisse de canard..."
13980d31-9002-457d-8d49-b451f08f473c,"Edulcorants sorbitol, isomalt, sirop de maltit...","Edulcorants sorbitol, isomalt, sirop de maltit..."
74297717-3fa8-4aed-95cc-e8737c1a6157,"sucre, amidon modifié, LACTOSÉRUM en poudre, d...","Z16005 / 002\nsucre, amidon modifié, LACTOSÉRU..."


On constitue une table pour intégration dans le rapport :

In [17]:
with pd.option_context("max_colwidth", 100000):
    tex_str = (
    sub_sample.sample(20, random_state=41)
              .replace(r'^\s*$', np.nan, regex=True)
              .to_latex(index=False,
                   index_names=False,
                   column_format='p{7cm}p{7cm}',
                   na_rep='<rien>',
                   longtable=True,
                   header=["Liste d'ingrédients cible", "Liste d'ingrédients prédite"],
                   label='tbl:GT_prediction_sample',
                   caption="Extrait des résultats de la prédiction",
                        
                  )
             .replace(r'\textbackslash n', r' \newline ')
             .replace(r'\\', r'\\ \hline')
    )

# with open(Path('..') / 'tbls' / 'GT_prediction_sample.tex', 'w') as file:
#     file.write(tex_str)